<a href="https://colab.research.google.com/github/marcelohpos/PROJETO-ICD/blob/master/icd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopy seaborn plotly --quiet # Para inserir o nome das cidades no df de acordo com as suas coordenadas

In [2]:
# Importar bibliotecas
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import time
import plotly.express as px
import plotly.graph_objects as go
import numpy as np # Adicionado para melhor manipulação, se necessário, mas não é estritamente obrigatório aqui

##**Análises Gerais:**

In [3]:
# -------------------------------
# Carregar dados
# -------------------------------

# 1️⃣ Carregar os dados
url = "https://raw.githubusercontent.com/marcelohpos/PROJETO-ICD/master/datasets/LA_daily_air_quality.csv"
dados_clima = pd.read_csv(url)

In [4]:
# Obter coordenadas únicas
coordenadas_unicas = dados_clima[['latitude', 'longitude']].drop_duplicates().reset_index(drop=True)

# Usar a API Nominatim (OpenStreetMap)
geolocator = Nominatim(user_agent="qualidade_ar_analise")
nomes = []

print("🔎 Consultando cidades... (pode demorar um pouco)")
for i, row in coordenadas_unicas.iterrows():
    try:
        location = geolocator.reverse((row['latitude'], row['longitude']), language='pt')
        if location and 'address' in location.raw:
            cidade = (location.raw['address'].get('city') or
                      location.raw['address'].get('town') or
                      location.raw['address'].get('state') or
                      "Desconhecida")
        else:
            cidade = "Desconhecida"
        nomes.append(cidade)
        print(f"→ {row['latitude']}, {row['longitude']} → {cidade}")
    except Exception as e:
        print(f"⚠️ Erro em {i}: {e}")
        nomes.append("Erro")

    time.sleep(1)  # evita bloqueio de requisições pela API

coordenadas_unicas["Cidade"] = nomes

# Juntar as cidades ao DataFrame original
dados_clima = dados_clima.merge(coordenadas_unicas, on=["latitude", "longitude"], how="left")

🔎 Consultando cidades... (pode demorar um pouco)
→ -34.6037, -58.3816 → Buenos Aires
→ -19.0196, -65.2619 → Sucre
→ -15.8267, -47.9218 → Brasília
→ -33.4378, -70.6504 → Região Metropolitana de Santiago
→ 4.711, -74.0721 → Bogotá
→ 9.9281, -84.0907 → San José
→ 23.1136, -82.3666 → La Habana
→ 18.4861, -69.9312 → São Domingos
→ -0.1807, -78.4678 → Quito
→ 13.6929, -89.2182 → São Salvador
→ 14.6349, -90.5069 → Cidade da Guatemala
→ 18.5944, -72.3074 → Porto Príncipe
→ 14.0723, -87.1921 → Tegucigalpa
→ 19.4326, -99.1332 → Cidade do México
→ 12.114, -86.2362 → Manágua
→ 8.9824, -79.5199 → Bella Vista
→ -25.2637, -57.5759 → Assunção
→ -12.0464, -77.0428 → Lima
→ -34.9011, -56.1645 → Montevidéu
→ 10.4806, -66.9036 → Caracas


In [5]:
dados_clima

,date,latitude,longitude,pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone,Cidade
0,2022-07-29 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN,Buenos Aires
1,2022-07-30 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN,Buenos Aires
2,2022-07-31 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN,Buenos Aires
3,2022-08-01 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN,Buenos Aires
4,2022-08-02 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN,Buenos Aires
...,...,...,...,...,...,...,...,...,...,...
12635,2024-04-16 00:00:00+00:00,10.4806,-66.9036,9.3,6.1,232.0,7.3,2.1,41.0,Caracas
12636,2024-04-17 00:00:00+00:00,10.4806,-66.9036,28.5,19.6,510.0,18.5,3.4,36.0,Caracas
12637,2024-04-18 00:00:00+00:00,10.4806,-66.9036,23.1,16.2,446.0,14.2,3.2,58.0,Caracas
12638,2024-04-19 00:00:00+00:00,10.4806,-66.9036,40.7,29.1,775.0,26.5,5.6,49.0,Caracas


In [6]:
dados_clima.describe()

,latitude,longitude,pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone
count,12640.000000,12640.000000,12520.000000,12520.000000,12520.000000,12520.000000,12520.000000,12520.000000
mean,-0.351840,-74.647240,20.669625,13.758530,414.904233,22.484069,8.930359,47.499201
std,19.473623,12.946283,16.881122,11.665622,496.742344,24.845234,13.674541,32.397686
min,-34.901100,-99.133200,0.300000,0.200000,58.000000,0.000000,0.000000,0.000000
25%,-16.624925,-84.627075,11.400000,7.500000,181.000000,6.700000,1.700000,26.000000
50%,9.455250,-75.557450,17.100000,11.200000,263.000000,15.100000,4.100000,42.000000
75%,14.212950,-66.493175,24.700000,16.500000,444.000000,30.900000,10.200000,62.000000
max,23.113600,-47.921800,292.400000,204.900000,7635.000000,324.400000,166.200000,332.000000


1- Como a qualidade do ar em Brasília se compara com a das capitais de outros países como Chile, Colômbia ou Argentina?

2- Gráficos de violino no que diz respeito a cada indicador de ar considerando cada capital

In [7]:
# Converter o DataFrame para formato longo (ideal para gráficos)
dados_longos = dados_clima.melt(
    id_vars=["Cidade"],
    value_vars=["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone"],
    var_name="Indicador",
    value_name="Valor"
)

# Garantir que só existam linhas válidas
dados_longos = dados_longos.dropna(subset=["Valor", "Cidade"])

In [8]:
# Criar o gráfico único com Dropdown (usando Plotly Graph Objects)

# Pegar a lista de indicadores e cidades
indicadores = dados_longos["Indicador"].unique()
cidades = dados_longos["Cidade"].unique()

# Definir uma paleta de cores para que as cidades mantenham a mesma cor
cores_cidades = {cidade: f'rgba({np.random.randint(0, 255)}, {np.random.randint(0, 255)}, {np.random.randint(0, 255)}, 0.8)'
                 for cidade in cidades}

# 1. Criar um trace (gráfico) para CADA PAR (Indicador, Cidade).
traces = []
all_traces_labels = [] # Lista para armazenar o rótulo de cada trace (Indicador_Cidade)

for indicador in indicadores:
    for cidade in cidades:
        df_par = dados_longos[(dados_longos['Indicador'] == indicador) &
                              (dados_longos['Cidade'] == cidade)]

        # Ignorar traces vazios
        if df_par.empty:
             continue

        # O nome/rótulo do trace será uma combinação dos filtros
        trace_label = f"{indicador}_{cidade}"
        all_traces_labels.append(trace_label)

        # Usamos go.Violin
        trace = go.Violin(
            y=df_par['Valor'],
            name=cidade, # O nome/legenda será a cidade
            customdata=df_par[['Cidade', 'Indicador']],
            # Definir a categoria no eixo X (será o INDICADOR no modo de seleção única)
            x=[indicador] * len(df_par),
            legendgroup=cidade,     # Agrupar cidades na legenda
            scalegroup=indicador,   # ESSENCIAL: Garante que as escalas dos violinos de um INDICADOR sejam comparáveis
            box_visible=True,       # Adiciona o Box Plot
            meanline_visible=True,  # Adiciona a linha da média
            points=False,           # Mostra os pontos individuais
            showlegend=(indicador == indicadores[0]), # Mostra a legenda apenas para o primeiro indicador
            # Inicialmente, mostramos apenas o primeiro Indicador
            visible=(indicador == indicadores[0]),
            marker_color=cores_cidades[cidade], # Usar cores consistentes por cidade
        )
        traces.append(trace)

# 2. Criar a Figure e adicionar todos os traces
fig = go.Figure(data=traces)

# 3. Criar os botões do Dropdown para o INDICADOR
buttons_indicador = []
for i, indicador_sel in enumerate(indicadores):
    # Visibilidade dos traces: visível se começar com o indicador selecionado
    visible = [trace_label.startswith(indicador_sel + "_") for trace_label in all_traces_labels]

    # Criar o dicionário do botão
    button = dict(
        label=indicador_sel,
        method='update',
        args=[
            {'visible': visible},  # Argumento 1: visibilidade dos traces
            {'title': f'Distribuição do indicador {indicador_sel} por cidade'} # Argumento 2: título
        ]
    )
    buttons_indicador.append(button)

# 4. Criar os botões do Dropdown para a CIDADE
buttons_cidade = []

# -------------------------------------------------------------------------
# BOTÃO DE "TODAS AS CIDADES" (MANTER VISIBILIDADE COMPLETA E ALTERAR LAYOUT X)
# -------------------------------------------------------------------------
# Quando este botão é clicado, queremos que o eixo X de cada violino
# visível seja a CIDADE, para que fiquem lado a lado.

all_cities_button = dict(
    label='Todas as Cidades',
    method='update',
    args=[
        # A visibilidade completa anula o filtro de cidade (o filtro de indicador continua ativo)
        {'visible': [True] * len(all_traces_labels)},
        # Redefinir o eixo X: O Plotly usará o 'name' (que é a Cidade) como categoria principal.
        {'xaxis': {'categoryorder': 'array', 'categoryarray': cidades.tolist(), 'title': 'Cidade'}},
    ]
)
buttons_cidade.append(all_cities_button)

# -------------------------------------------------------------------------
# BOTÕES DE CIDADES INDIVIDUAIS (Filtragem e retorno ao Layout X do Indicador)
# -------------------------------------------------------------------------
for i, cidade_sel in enumerate(cidades):
    # Um trace é visível se for DESSA CIDADE
    visible = [trace_label.endswith("_" + cidade_sel) for trace_label in all_traces_labels]

    # Criar o dicionário do botão
    button = dict(
        label=cidade_sel,
        method='update',
        args=[
            {'visible': visible}, # Argumento 1: visibilidade dos traces
            # Ao selecionar uma ÚNICA cidade, voltamos a agrupar todos os traces sob
            # a única categoria do Indicador (mantido pelo botão de Indicador)
            # Para que o layout funcione, usamos a primeira cidade como array, mas o violino
            # se posicionará no Indicador devido ao seu atributo 'x'.
            {'xaxis': {'categoryorder': 'array', 'categoryarray': [cidades[0]], 'title': 'Cidade'}},
        ]
    )
    buttons_cidade.append(button)

# 5. Adicionar os dois dropdowns ao layout do gráfico
fig.update_layout(
    updatemenus=[
        # Dropdown do Indicador (Controla qual grupo de traces é visível)
        dict(
            type="dropdown",
            direction="down",
            x=0.0,
            y=1.1,
            showactive=True,
            buttons=buttons_indicador,
            pad={"r": 10, "t": 10}
        ),
        # Dropdown da Cidade (Filtro e controle do modo "lado a lado" vs "agrupado")
        dict(
            type="dropdown",
            direction="down",
            x=0.25,
            y=1.1,
            showactive=True,
            buttons=buttons_cidade,
            pad={"r": 10, "t": 10}
        )
    ],
    title=f'Distribuição do indicador {indicadores[0]} por cidade',
    # Configuração inicial do eixo X (no modo de seleção única de indicador)
    xaxis=dict(
        categoryorder='array',
        categoryarray=[indicadores[0]],
        title='Cidade (Filtre acima)'
    ),
    yaxis_title='Valor (μg/m³ ou ppb)',
    violingroupgap=0.2, # Espaçamento entre os violinos (cidades)
    violingap=0.05,     # Espaçamento dentro do grupo
    width=1200,
    height=600,
    violinmode='group' # Garante que os violinos sejam desenhados lado a lado, em vez de sobrepostos
)

# 6. Exibir o gráfico
fig.show()

3- fazer um calendário de heatmap pros niveis de ar e olhar se tem alguma sazonalidade de brasilia ou geral de tudo

4- Describe geral da tabela, mostrando os dados que temos e algumas caracteristicas gerais deles

5- visualização geral dos indices, com mapa interativo da seria temporal mostrando a evolução da temp, da evapotranspiração e do vento

tentar juntar isso com uma visualização geral das variaveis

6- estruturar uma ideia pra medir a qualidade do ar

##**Intervalo de Confiança**

1- A média dos níveis de poluição em Brasília difere de forma estatisticamente significativa da média das outras capitais da América Latina?

2- avaliar correlações entre os gases e as variavies do tempo


##**Regressão:**

1- É possível desenvolver um modelo de regressão que preveja os níveis de PM2.5 para cidades da América Latina, usando dados meteorológicos e históricos como entrada?

2- Quais variáveis meteorológicas (temperatura, velocidade do vento, pressão atmosférica) são os preditores mais importantes para a qualidade do ar, e essa importância varia entre as diferentes regiões ?